1. Import data from a file
2. Missing Data
3. Formatting the Data for XGBoost
4. Building a preliminary XGBoost Model
5. Optimize Parameters with Cross Validation and GridSearch()
6. Build, Drawing, Interpreting and Evaluating the optimized XGBoost Model


In [1]:
import pandas as pd #data and one-hot encoding
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split #split the data for training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer #for evulation
from sklearn.model_selection import GridSearchCV #cross validation
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import plot_confusion_matrix


ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/Users/yuxianwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/__init__.py)

In [ ]:
df = pd.read_csv('xxxx.csv')

In [ ]:
df.head()

In [ ]:
df.drop(['x1','x2','x3','x4'], axis = 1,inplace = True) #drop columns
#axis = 0 rows, axis =1 columns 
df.head()

In [ ]:
df['Count'].unique()
df['Country'].unique()
df['State'].unique()

We cant have wightspace if we want to draw a tree. we need to replace them.


In [ ]:
df['City'].replace(' ','_', regex = True, inplace = True)
#regex 是否将替换操作视作正则表达式regular expression. 
#因此所有单个空格string将会被下划线取代

In [ ]:
df.columns = df.columns.str.replace(' ','_')
#all columns

Now we finish the missing data removing. Now we are ready to identify and deal with Missing Data 

Part1. Identifying Missing Data

In [2]:
df.dtypes

NameError: name 'df' is not defined

Part2. Deal with the missing data

In [ ]:
len(df.loc[df['x1'] ==' '])
#how many rows are missing data

In [ ]:
df.loc[df['x1'] == ' ']

In [ ]:
df.loc[df['x1'] == ' ', 'x1'] = 0
#让空格的值等于0
df['Total_Charges'] = df['Total_Charges'].apply(lambda x: 0 if x == ' ' else x)
#这可以达到一样的效果

In [ ]:
df['x1'] = pd.to_numeric(df['x1']) 
#convert into number

Format Data

In [ ]:
X = df.drop('target_value',axis=1).copy()
#alternatively X=df_no_misssing.iloc[:,:-1]

In [ ]:
y = df['target_value'].copy()

In [3]:
#one-hot encoding

In [ ]:
X.dtypes

In [ ]:
pd.get_dummies(X,columns = ['x2']).head()

In [ ]:
X_encoded = pd.get_dummies(X, columns = ['x2','x3','x4'])
X_encoded.head()
#all the columns need to be one-hot encoding

Build A Preliminary XGBoost Tree
First, let's observe that this data is imbalanced by dividing the number of people who left the company, where y =1, by the number of people in the dataset

In [ ]:
sum(y)/len(y)
#27% of people left the company

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, 
                                                    y, 
                                                    random_state = 42, 
                                                    stratify = y)
#stratify=y确保了training set和testing set中y的分布是保持一致的

In [ ]:
clf_xgb = xgb.XGBclassifier(objective = 'binary:logistic', 
                            missing = None,
                            seed = 42)
#XGBoost会将任务视为二分类问题，并使用逻辑回归作为损失函数。输出的结果是一个介于0和1之间的概率值，表示样本属于正类的概率。
clf.xgb.fit（X_train,
        y_train,
        verbose = True,  #训练过程中每一轮（每一次迭代）的结果都会被打印出来。
        early_stopping_round=10, 
        eval_meric ='aucpr',#“Precision-Recall AUC”，即精确率-召回率曲线下的面积。这是一个常用于不平衡分类问题的评估指标。
        eval_set=[(X_test,y_test)]

Evaluation 

In [ ]:
plot_confusion_martix(clf_xgb,
                     X_test,
                     y_test,
                     values_format = 'd',
                     display_labels = ['Did not leave','left'])

Optimize Parametes using Cross Validation and GridSearch()


In [ ]:
param_grid = {
    'max_depth':[3,4,5],
    'learning_rate':[0.1,0.01,0.05],
    'gamma':[0,0.25,1.0]
    'reg_lambda':[0,1.0,10.0],
    'scale_pos_weight':[1,3,5]
}

optimal_params=GridSearchCV(
    estimator=xgb.XGBClassifier(objective ='binary:logistic',
                               seed =42,
                               subsample =0.9,#每棵树训练时使用的数据占比
                               colsample_bytree=0.5),#每棵树训练时随机选择的特征占比
    param_grid = param_grid
    scoring = 'roc_auc',
    verbose=0 #控制输出信息的详细程度。0表示不输出。
    n_jobs=10 #并行运行的作业数量。10表示使用10个CPU核进行计算
    cv=3 #交叉验证的折数。3表示使用3折交叉验证
)

Build, Evaluating,Drawing,and Interpreting the optimal model

In [ ]:
clf_xgb=xgb.XGBClassifier(
    seed=42,
    objective = 'binary:logistic',
    gamma=0.25,
    learn_rate=0.1,
    max_depth=4,
    reg_lambda=10,
    scale_pos_weight=3,#在XGBoost中用于处理类别不平衡问题，通过调整正类样本的权重，确保模型在训练过程中能够更加重视少数类样本，从而提高模型在不平衡数据集上的表现。设置该参数时，通常根据数据集中正类和负类样本的比例来确定其值。
    subsample=0.9,
    colsample_bytree=0.5
)

clf_xgb.fit(X_train,
           y_train,
           verbose=True,
           early_stopping_round=10,
           eval_meteric='aucpr',
           eval_set = [(X_test,y_test)])

In [ ]:
plot_confusion_matrix(
    clf_xgb,
    X_test,
    y_test,
    values_format='d',
    display_labels = ['Did not leave','left'])

Draw the first XGBoost Tree

In [ ]:
clf_xgb=xgb.XGBClassifier(
    seed=42,
    objective = 'binary:logistic',
    gamma=0.25,
    learn_rate=0.1,
    max_depth=4,
    reg_lambda=10,
    scale_pos_weight=3,#在XGBoost中用于处理类别不平衡问题，通过调整正类样本的权重，确保模型在训练过程中能够更加重视少数类样本，从而提高模型在不平衡数据集上的表现。设置该参数时，通常根据数据集中正类和负类样本的比例来确定其值。
    subsample=0.9,
    colsample_bytree=0.5
)

clf_xgb.fit(X_train,
           y_train)

In [ ]:
bst= clf_xgb.get_booster()
for importance_type in ('weight','gain','cover','tptal_gain','total_cover'):
    print('%s:'%importance_type,bsg.get_score(imporantce_type=importance_type))
    
    

In [ ]:
node_params = {
    'shape'='box',
    'style':'filled,rounded',
    'fillcolor':'#78cbe'
}

leaf_params={
    'shape'='box',
    'style':'filled',
    'fillcolor':'#e48038'
}

In [ ]:
xgb.to_graphviz(clf_xgb,num_trees=0,size="10,10",
               confition_node_params=node_params,
               leaf_node_params=leaf_params)